# File Name: simple_prompt_mgmt.ipynb
### Location: Chapter 16
### Purpose: 
#####       1. Create Start and End node of prompt flow
#####       2. Create invoke and eval prompt and prompt version using Prompt management 
#####        3. Create evaluate and invoke node of prompt flow 
#####        4. Create connections between nodes
#####        5. Create prompt flow with nodes and connections 
#####        6. Prepare the prompt flow and find out the status of the prompt flow
#####        7. Create prompt flow version
#####        8. Create prompt alias
#####        9. Testing the prompt flow 
#####        10. Deleting resources associated with an AWS Bedrock prompt flow, including aliases, versions, the flow itself, and associated prompts.
        
##### Dependency: Not Applicable
# <ins>-----------------------------------------------------------------------------------</ins>

# <ins>Amazon SageMaker Classic</ins>
#### Those who are new to Amazon SageMaker Classic. Follow the link for the details. https://docs.aws.amazon.com/sagemaker/latest/dg/studio.html

# <ins>Environment setup of Kernel</ins>
##### Fill "Image" as "Data Science"
##### Fill "Kernel" as "Python 3"
##### Fill "Instance type" as "ml-t3-medium"
##### Fill "Start-up script" as "No Scripts"
##### Click "Select"

###### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/notebooks-create-open.html for details.

# <ins>Mandatory installation on the kernel through pip</ins>

##### This lab will work with below software version. But, if you are trying with latest version of boto3, awscli, and botocore. This code may fail. You might need to change the corresponding api. 

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell. 

In [ ]:
%pip install --no-build-isolation --force-reinstall -q \
    "boto3" \
    "awscli" \
    "botocore" 

# <ins>Disclaimer</ins>

##### You will see pip dependency errors. you can safely ignore these errors and continue executing rest of the cell.

# <ins>Restart the kernel</ins>

In [ ]:
# restart kernel
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

# <ins>Python package import</ins>

##### boto3 offers various clients for Amazon Bedrock to execute various actions.
##### botocore is a low-level interface to AWS tools, while boto3 is built on top of botocore and provides additional features

In [ ]:
import json
import os
import boto3
import botocore
import warnings
import time
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError

### Ignore warning 

In [ ]:
warnings.filterwarnings('ignore')

## Define important environment variable

In [ ]:
# Try-except block to handle potential errors
try:
    # Create a new Boto3 session to interact with AWS services
    # This session is responsible for managing credentials and region configuration
    boto3_session = boto3.session.Session()

    # Retrieve the current AWS region from the session (e.g., 'us-east-1', 'us-west-2')
    aws_region_name = boto3_session.region_name
    
    # Initialize Bedrock and Bedrock Runtime and Bedrock Agent clients using Boto3
    # These clients will allow interactions with Bedrock-related AWS services
    boto3_bedrock_client = boto3.client('bedrock', region_name=aws_region_name)
    boto3_bedrock_runtime_client = boto3.client('bedrock-runtime', region_name=aws_region_name)
    boto3_bedrock_agent_client = boto3.client(service_name="bedrock-agent", region_name=aws_region_name)
    boto3_bedrock_agent_runtime_client = boto3.client(service_name="bedrock-agent-runtime", region_name=aws_region_name)
    
    
    # Define the name of the invoke prompt 
    invoke_prompt_name = "iPhone_accessories_recommendation"
    invoke_prompt_description="Initial prompt for iPhone accessories recommendation"  # Description of the prompt
    
    # Define the name of the eval prompt 
    eval_prompt_name = "eval_iPhone_accessories_recommendation"
    eval_prompt_description="Eval prompt for iPhone accessories recommendation"  # Description of the prompt
    
    # Define the name of the prompt flow
    flow_name = "Flow_iPhone_accessories_recommendation"
    flow_description="Prompt flow for iPhone accessories recommendation"  # Description of the prompt flow
    
    ### Adjust with your preferred model IDs for invocations and evaluation - Note some models are only available in certain regions:
    bedrock_model_invoke_id = "amazon.titan-text-express-v1"
    bedrock_model_eval_id = "anthropic.claude-3-haiku-20240307-v1:0"

    # Store all relevant variables in a dictionary for easier access and management
    variables_store = {
        "aws_region_name": aws_region_name,                          # AWS region name
        "boto3_bedrock_client": boto3_bedrock_client,                # Bedrock client instance
        "boto3_bedrock_runtime_client": boto3_bedrock_runtime_client,  # Bedrock Runtime client instance
        "boto3_bedrock_agent_client": boto3_bedrock_agent_client,  # Bedrock agent client instance
        "boto3_bedrock_agent_runtime_client": boto3_bedrock_agent_runtime_client,  # Bedrock agent client instance
        "invoke_prompt_name": invoke_prompt_name,
        "invoke_prompt_description": invoke_prompt_description,
        "eval_prompt_name": eval_prompt_name,
        "eval_prompt_description": eval_prompt_description,
        "flow_name": flow_name,
        "flow_description": flow_description,
        "bedrock_model_invoke_id": bedrock_model_invoke_id,
        "bedrock_model_eval_id": bedrock_model_eval_id,
        "boto3_session": boto3_session                               # Current Boto3 session object
    }

    # Print all stored variables for debugging and verification
    for var_name, value in variables_store.items():
        print(f"{var_name}: {value}")

# Handle any exceptions that occur during the execution
except Exception as e:
    # Print the error message if an unexpected error occurs
    print(f"An unexpected error occurred: {e}")

# Find out ARN of IAM role for Flows Service Role 

### list_filtered_iam_roles(): Retrieves and filters AWS IAM roles based on specific conditions. This is required for executing the prompt flow. 

In [ ]:
def list_filtered_iam_roles():
    """
    Lists IAM roles in the AWS account that match specific ARN patterns
    and do not contain 'Lambda' or 'Stack', with error handling.
    """
    try:
        # Create an IAM client
        iam_client = boto3.client('iam')
        
        # Retrieve the list of IAM roles
        roles = iam_client.list_roles()

        # Define the keywords to include and exclude
        include_keywords = ["genalbookbedrocksagemaker"]
        exclude_keywords = ["Lambda", "Stack"]

        # Initialize a variable to store the matching role ARN
        flows_service_role = None

        # Iterate through the roles and filter based on the ARN
        for role in roles['Roles']:
            arn = role['Arn']
            # Check for inclusion and exclusion conditions
            if (any(keyword.lower() in arn.lower() for keyword in include_keywords) and
                    not any(keyword.lower() in arn.lower() for keyword in exclude_keywords)):
                flows_service_role = arn  # Store the matching ARN

        if flows_service_role:
            return flows_service_role
        else:
            print("No matching roles found.")
            return None

    except NoCredentialsError:
        print("Error: AWS credentials not found.")
    except PartialCredentialsError:
        print("Error: AWS credentials are incomplete.")
    except ClientError as e:
        print(f"Client error occurred: {e.response['Error']['Message']}")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")
    return None


flows_service_role = list_filtered_iam_roles()
if flows_service_role:
    print(f"Found Matching Role ARN: {flows_service_role}")
else:
    print("No valid role ARN found.")

# Prompt Flow Architecture Diagram 

<img src="./prompt_flow_arc_diagram.png" style="width: 600px; height: 400px;">

# Create start node 
#### Refer: Architecture diagram step (1) above 

##### The create_start_node function defines the "Start" node for a flow, which is an input node that initiates the process. It includes a configuration for the input (an empty object) and outputs a string named document. 

In [ ]:
# Function to define the Start node
def create_start_node():
    try:
        # Start node definition: This node is used to define the input for the flow
        return {
            "name": "Start_Node",  # The name of the node
            "type": "Input",  # The type of node (Input node to start the flow)
            "configuration": {
                "input": {}  # Configuration for input, here it's just an empty object
            },
            "outputs": [  # The outputs for this node
                {
                    "name": "document",  # Output name
                    "type": "String"  # Output type (String, in this case)
                }
            ],
        }
    except Exception as e:
        # Catching any exceptions that occur while defining the node
        print(f"Error in creating Start node: {e}")
        return None  # Returning None in case of an error

# Create end node 
#### Refer: Architecture diagram step (4) above 

##### The create_end_node function defines the configuration for the "End" node in a flow, which represents the final step in processing. This node takes the input data, processes it, and produces the final output. The node is configured as an "Output" type with no specific output configuration, and it expects input data under the name "response", using the expression $.data to refer to the flow's data. 

In [ ]:
# Function to define the End node with error handling
def create_end_node():
    try:
        # The "End" node represents the final step in the flow.
        # It receives input data and outputs the result.
        return {
            "name": "End_Node",  # Name of the node, representing the end of the flow.
            "type": "Output",  # The type of node, indicating it's an output node.
            "configuration": {
                "output": {}  # No specific configuration for output, it's the result of the flow.
            },
            "inputs": [
                {
                    "expression": "$.data",  # Expression for the input, referring to data flowing in the flow.
                    "name": "document",  # The name of the input variable.
                    "type": "String"  # The expected data type for this input.
                }
            ],
        }
    except Exception as e:
        # Catch any exceptions that occur during the creation of the End node
        print(f"An error occurred while creating the End node: {e}")
        return None  # Return None if an error occurs, indicating failure to create the node.


# Create invoke and eval prompt using Prompt management one by one. 

#### Details is in Chapter 15 simple_prompt_mgmt.ipynb to Create prompt and prompt version using Prompt management

##### 1. Create a eval prompt using Prompt management
##### 2. Create a version of a eval prompt in Prompt management  

In [ ]:
try:
    # Step 1: Create a new prompt
    response = boto3_bedrock_agent_client.create_prompt(
        name=eval_prompt_name,
        description=eval_prompt_description,
        variants=[
            {
                "inferenceConfiguration": {
                    "text": {
                        "maxTokens": 2000,
                        "temperature": 0,
                    }
                },
                "modelId": bedrock_model_eval_id,
                "name": "eval_variant",
                "templateConfiguration": {
                    "text": {
                        "inputVariables": [
                            {"name": "input"},
                            {"name": "output"}
                        ],
                        "text": """
                        You're tasked with evaluating the prompts and answers generated by an generative AI model. The input prompt is enclosed within the <input> tags, the output response within the <output> tags, the evaluation criteria for the prompt within <prompt_criteria> tags, and the criteria for the answer within <answer_criteria> tags.

                        <input> {{input}} </input> <output> {{output}} </output>

                        <prompt_criteria>
                        The prompt must be clear, precise, and detailed.
                        The prompt's objective should be well-defined, grammatically correct, and easy to understand.
                        Including examples improves the quality of the prompt.
                        Setting a role or providing context makes the prompt more effective.
                        Adding details about the expected answer's format and tone is beneficial. 
                        </prompt_criteria>

                        <answer_criteria>
                        The answers must be accurate, well-structured, and technically complete.
                        The answers must avoid hallucinations, fabricated content, or offensive language.
                        The answers must adhere to proper grammar rules.
                        The answers must align entirely with the instruction or question given in the prompt. 
                        </answer_criteria>

                        Evaluate the response generated by the AI model within the <output> tags with a score ranging from 0 to 100, based on the <answer_criteria>. Even minor hallucinations should significantly affect the evaluation score. Similarly, assess the input prompt within the <input> tags using a score from 0 to 100 based on the <prompt_criteria>.

                        Provide your evaluation as a JSON object containing:

                        An 'answer-score' key with the answer's evaluation score.
                        A 'prompt-score' key with the prompt's evaluation score.
                        A 'justification' key with a detailed explanation for the scores, highlighting any specific issues such as hallucinations or grammatical errors.
                        An 'input' key containing the content of the <input> tags.
                        An 'output' key containing the content of the <output> tags.
                        A 'prompt-recommendations' key with actionable suggestions for improving the prompt based on your evaluations.
                        Ensure your response includes only the JSON object without any preamble or additional text.
                        """
                    }
                },
                "templateType": "TEXT"
            }
        ],
        defaultVariant="eval_variant"
    )

    # Extract key details from the response
    promptEvalId = response["id"]
    promptEvalArn = response["arn"]
    promptEvalName = response["name"]

    # Display the extracted details
    print(f"Prompt ID: {promptEvalId}\nPrompt ARN: {promptEvalArn}\nPrompt Name: {promptEvalName}")

    # Step 2: Create a new prompt version
    version_response = boto3_bedrock_agent_client.create_prompt_version(
        promptIdentifier=promptEvalId
    )

    
    # Extracting version from the versioned response
    eval_versioned_version = version_response.get("version", "Unknown")
    print(f"Versioned version: {eval_versioned_version}")

except NoCredentialsError:
    print("Error: AWS credentials were not found. Please configure your credentials.")
except PartialCredentialsError:
    print("Error: Incomplete AWS credentials provided. Check your configuration.")
except ClientError as e:
    # Handle specific AWS client errors
    print(f"Client error occurred: {e.response['Error']['Message']}")
except Exception as e:
    # Handle any other unexpected exceptions
    print(f"An unexpected error occurred: {str(e)}")

##### 1. Create a invoke prompt using Prompt management
##### 2. Create a version of a invoke prompt in Prompt management  

In [ ]:
try:
    # Step 1: Create a new prompt
    response = boto3_bedrock_agent_client.create_prompt(
        name=invoke_prompt_name,
        description=invoke_prompt_description,
        variants=[
            {
                "inferenceConfiguration": {
                    "text": {
                        "maxTokens": 2000,
                        "temperature": 0,
                    }
                },
                "modelId": bedrock_model_invoke_id,
                "name": "invoke_variant",
                "templateConfiguration": {
                    "text": {
                        "inputVariables": [
                            {"name": "input"}
                        ],
                        "text": "{{input}}"
                    }
                },
                "templateType": "TEXT"
            }
        ],
        defaultVariant="invoke_variant"
    )

    # Extract key details from the response
    promptInvokeId = response["id"]
    promptInvokeArn = response["arn"]
    promptInvokeName = response["name"]

    # Display the extracted details
    print(f"Prompt ID: {promptInvokeId}\nPrompt ARN: {promptInvokeArn}\nPrompt Name: {promptInvokeName}")

    # Step 2: Create a new prompt version
    version_response = boto3_bedrock_agent_client.create_prompt_version(
        promptIdentifier=promptInvokeId
    )
    
    # Extracting version from the versioned response
    invoke_versioned_version = version_response.get("version", "Unknown")
    print(f"Versioned version: {invoke_versioned_version}")

except NoCredentialsError:
    print("Error: AWS credentials were not found. Please configure your credentials.")
except PartialCredentialsError:
    print("Error: Incomplete AWS credentials provided. Check your configuration.")
except ClientError as e:
    # Handle specific AWS client errors
    print(f"Client error occurred: {e.response['Error']['Message']}")
except Exception as e:
    # Handle any other unexpected exceptions
    print(f"An unexpected error occurred: {str(e)}")

# Create eval node 
#### Refer: Architecture diagram step (3) above 

##### Attaching eval prompt with evaluate node
##### The create_evaluate_node() function defines a modular "Evaluate" node within a prompt flow. It constructs a dictionary representing the node's structure, including its name, type, input and output configurations, and a reference to a predefined prompt ARN.

In [ ]:
# Function to define the Evaluate node
def create_evaluate_node():
    try:
        # Attempt to define the Evaluate node
        evaluate_node = {
            "name": "Evaluate_Node",  # The name of the node
            "type": "Prompt",  # The type of the node, in this case, a prompt
            "configuration": {
                "prompt": {
                    "sourceConfiguration": {
                        "resource": {
                            "promptArn": promptEvalArn  # Reference to the prompt ARN (Replace with your ARN)
                        }
                    }
                }
            },
            "inputs": [
                {
                    "expression": "$.data",  # The expression pointing to the input data
                    "name": "input",  # The name of the input variable
                    "type": "String"  # The type of the input data
                },
                {
                    "expression": "$.data",  # Another expression pointing to the output data from previous nodes
                    "name": "output",  # The name of the output variable
                    "type": "String"  # The type of the output data
                }
            ],
            "outputs": [
                {
                    "name": "modelCompletion",  # The name of the output from this node
                    "type": "String"  # The type of the output (string in this case)
                }
            ],
        }

        # Return the created evaluate node structure
        return evaluate_node

    except Exception as e:
        # Catch any errors that occur while defining the node
        print(f"An error occurred while creating the Evaluate node: {e}")
        return None

# Create invoke node 
#### Refer: Architecture diagram step (2) above 

##### Attaching invoke prompt with invoke node
##### The create_invoke_node() function defines a modular "Invoke" node within a prompt flow, which invokes a model using a predefined prompt ARN (promptInvokeArn). It accepts input data in the form of a string, processes it through the model, and outputs the model's response as a string.

In [ ]:
# Function to define the Invoke node
def create_invoke_node():
    """
    # Create Invoke Node for Prompt-based Interaction
    #### Purpose: This function defines an 'Invoke' node that will be part of a larger prompt flow.
    #### It includes configuration for a Prompt node that utilizes a predefined Prompt ARN (promptInvokeArn).
    #### The node is responsible for taking input data, invoking a model, and providing output data.
    
    ##### Workflow:
    1. The `Invoke_Node` uses a `Prompt` type to invoke an LLM-based model.
    2. The node references an existing prompt using its `promptArn`.
    3. It takes input from the `Start` node and sends output to the next node in the flow.
    """
    try:
        # Return the structure of the Invoke node
        return {
            "name": "Invoke_Node",  # Name of the node within the flow
            "type": "Prompt",  # Node type, indicating it invokes a prompt (LLM model)
            "configuration": {
                "prompt": {
                    "sourceConfiguration": {
                        # Referencing an existing prompt via its ARN
                        "resource": {
                            "promptArn": promptInvokeArn  # Replace with actual ARN of the prompt
                        }
                    }
                }
            },
            "inputs": [
                {
                    "expression": "$.data",  # Expression to access input data from previous nodes
                    "name": "input",  # Name of the input variable expected by the model
                    "type": "String"  # Data type for input
                }
            ],
            "outputs": [
                {
                    "name": "modelCompletion",  # Output name of the model's response
                    "type": "String"  # Data type for output (in this case, a string response from the model)
                }
            ],
        }
    except Exception as e:
        # Catch any exceptions during node creation and print an error message
        print(f"An error occurred while creating the Invoke node: {e}")
        return None  # Returning None if an error occurs during node creation


# Create connections between nodes
#### Refer: Architecture diagram above 

##### The create_connections() function defines the data flow connections between nodes in a prompt flow. It specifies three main connections: from the "Start" node to the "Invoke" node, from the "Invoke" node to the "Evaluate" node, and from the "Evaluate" node to the "End" node. Each connection defines the source and target nodes, along with the data mapping between them, specifying how the output from one node is passed as input to another.

In [ ]:
# Function to define the connections between nodes
def create_connections():
    """
    This function defines the connections between nodes in a modular prompt flow.
    Each connection maps the output of one node to the input of another, enabling data flow between components.

    Returns:
        list: A list of dictionaries representing the connections between nodes.
    """
    try:
        # Connection from Start_Node to Invoke_Node
        start_to_invoke = {
            "name": "start_2_invoke_connection",
            "source": "Start_Node",
            "target": "Invoke_Node",
            "type": "Data",
            "configuration": {
                "data": {
                    "sourceOutput": "document",
                    "targetInput": "input"
                }
            },
        }
        # Connection from Invoke_Node to Evaluate_Node
        invoke_to_evaluate = {
            "name": "invoke_2_evaluate_connection",
            "source": "Invoke_Node",
            "target": "Evaluate_Node",
            "type": "Data",
            "configuration": {
                "data": {
                    "sourceOutput": "modelCompletion",
                    "targetInput": "output"
                }
            },
        }
        # Connection from Start_Node to Evaluate_Node
        start_to_evaluate = {
            "name": "start_2_evaluate_connection",
            "source": "Start_Node",
            "target": "Evaluate_Node",
            "type": "Data",
            "configuration": {
                "data": {
                    "sourceOutput": "document",
                    "targetInput": "input"
                }
            },
        }
        # Connection from Evaluate_Node to End_Node
        evaluate_to_end = {
            "name": "evaluate_2_end_connection",
            "source": "Evaluate_Node",
            "target": "End_Node",
            "type": "Data",
            "configuration": {
                "data": {
                    "sourceOutput": "modelCompletion",
                    "targetInput": "document"
                }
            },
        }
        
        # Returning the list of all connections
        return [start_to_invoke, invoke_to_evaluate, start_to_evaluate, evaluate_to_end]
    except Exception as e:
        # Handling any exceptions that occur during connection creation
        print(f"Error in creating connections: {e}")
        return None


# Create prompt flow with nodes and connections
#### Refer: Architecture diagram above 

##### The create_flow() function is responsible for defining and creating a prompt flow using the AWS Bedrock service. It first constructs a flow definition that includes node configurations (Start, End, Invoke, and Evaluate nodes) and the data flow connections between them. The function attempts to create the flow by calling the create_flow API from the Bedrock agent client, and upon successful creation, it extracts and prints the flow's ID, ARN, and name.

In [ ]:
# Function to create the flow
def create_flow():
    try:
        # Create the flow definition with nodes and connections
        flow_definition = {
            "name": flow_name,
            "description": flow_description,
            "executionRoleArn": flows_service_role,
            "definition": {
                "nodes": [
                    create_start_node(),
                    create_end_node(),
                    create_invoke_node(),
                    create_evaluate_node()
                ],
                "connections": create_connections()
            }
        }


        # Create the flow
        response = boto3_bedrock_agent_client.create_flow(**flow_definition)

        # Handle the response
        flowEvalId = response["id"]
        flowEvalArn = response["arn"]
        flowEvalName = response["name"]
        print(f"Flow ID: {flowEvalId}\nFlow ARN: {flowEvalArn}\nFlow Name: {flowEvalName}")

        # Returning the flow response for further usage if necessary
        return ( flowEvalId, flowEvalArn, flowEvalName )

    except Exception as e:
        print(f"An error occurred while creating the flow: {e}")
        return None


# Ensure the flow creation result is valid before unpacking
flow_response = create_flow()

if flow_response:
    flowEvalId, flowEvalArn, flowEvalName = flow_response
else:
    print("Flow creation failed.")

# Prepare the prompt flow and find out the status of the prompt flow 

In [ ]:
# Function to prepare a flow and check its status
def prepare_flow(flow_eval_id):
    try:
        # Prepare the flow using the provided flow ID
        response = boto3_bedrock_agent_client.prepare_flow(
            flowIdentifier=flow_eval_id
        )
        
        # Extract and print the flow status
        flow_status = response["status"]
        print(f"Flow ID: {flow_eval_id}\nStatus: {flow_status}")
        
        # Return the flow status for further use
        return flow_status

    except Exception as e:
        # Handle and log any errors that occur during the preparation
        print(f"An error occurred while preparing the flow: {e}")
        return None


# Example usage
flow_status = prepare_flow(flowEvalId)

# Create prompt flow version 

In [ ]:
# Function to create a new flow version using the Bedrock agent
try:
    # Attempt to create a new version of the flow using the provided flowEvalId
    response = boto3_bedrock_agent_client.create_flow_version(
            flowIdentifier=flowEvalId
        )
    
    status_response = response["status"]
    status_version = response["version"]
        
    print(f"Version: {status_version}\nStatus: {status_response}")

except Exception as e:
    # Handle any exceptions that occur during the flow version creation
    print(f"Error occurred while creating flow version for flow ID {flowEvalId}: {e}")


# Create prompt alias

In [ ]:
# Function to create an alias for a flow
try:
    # Attempt to create an alias for the flow
    response = boto3_bedrock_agent_client.create_flow_alias(
        flowIdentifier=flowEvalId,
        name=flowEvalName,
        description="Alias Flow_iPhone_accessories_recommendation",
        routingConfiguration=[
            {
                "flowVersion": "1"
            }
        ]
    )

    # Print the response in a readable JSON format
    print(json.dumps(response, indent=2, default=str))

    # Extract the alias ID from the response
    flowEvalAliasId = response['id']

except Exception as e:
    # Handle any exceptions that occur during alias creation
    print(f"Error occurred while creating alias for flow ID {flowEvalId}: {e}")

# Testing the prompt flow 

In [ ]:
def execute_flow(prompt):
    """
    Function to execute a Bedrock flow using a prompt and process the streamed response.

    Parameters:
        prompt (str): The prompt to be sent to the flow.

    Returns:
        None: The response content is printed during the streaming process.
    """
    try:
        # Ensure flow and alias identifiers are defined
        if not flowEvalId or not flowEvalAliasId:
            raise ValueError("Flow ID or Flow Alias ID is not defined. Ensure they are initialized.")

        # Invoke the flow with the provided prompt
        response = boto3_bedrock_agent_runtime_client.invoke_flow(
            flowIdentifier=flowEvalId,
            flowAliasIdentifier=flowEvalAliasId,
            inputs=[
                { 
                    "content": { 
                        "document": prompt  # Pass the prompt to the flow
                    },
                    "nodeName": "Start_Node",  # Match the node name in the flow definition
                    "nodeOutputName": "document"  # Match the output name from the Start node
                }
            ]
        )

        # Stream the response events
        event_stream = response["responseStream"]
        print("Processing flow response stream...")

        # Iterate through each event in the stream
        for event in event_stream:
            # Check for flow output events and print the document content
            if "flowOutputEvent" in event and "content" in event["flowOutputEvent"]:
                document = event["flowOutputEvent"]["content"].get("document", "No document found")
                print(f"Flow Output: {document}")
            else:
                print("Unexpected event format:", event)

    except KeyError as e:
        print(f"KeyError: Missing expected key in response or event: {e}")
    except Exception as e:
        print(f"An error occurred during flow execution: {e}")

# Test the function with a prompt
prompt = "Recommend three popular accessories under $75 for a school student who recently bought an iPhone."
execute_flow(prompt)

# Clean up steps 

#### deleting resources associated with an AWS Bedrock prompt flow, including aliases, versions, the flow itself, and associated prompts.

In [ ]:
def delete_flow_alias(flow_id, alias_id):
    """
    Deletes a specific flow alias.

    Parameters:
        flow_id (str): Identifier for the flow.
        alias_id (str): Identifier for the alias to delete.
    """
    try:
        print("Deleting prompt flow alias...")
        response = boto3_bedrock_agent_client.delete_flow_alias(
            flowIdentifier=flow_id,
            aliasIdentifier=alias_id
        )
        print(json.dumps(response, indent=2, default=str))
        print("Prompt flow alias deletion completed.\n")
    except Exception as e:
        print(f"Error while deleting prompt flow alias: {e}\n")

def delete_flow_version(flow_id, version):
    """
    Deletes a specific version of a flow.

    Parameters:
        flow_id (str): Identifier for the flow.
        version (str): Version of the flow to delete.
    """
    try:
        print("Deleting prompt flow version...")
        response = boto3_bedrock_agent_client.delete_flow_version(
            flowIdentifier=flow_id,
            flowVersion=version
        )
        print(json.dumps(response, indent=2, default=str))
        print("Prompt flow version deletion completed.\n")
    except Exception as e:
        print(f"Error while deleting prompt flow version: {e}\n")

def delete_flow(flow_id):
    """
    Deletes an entire flow.

    Parameters:
        flow_id (str): Identifier for the flow.
    """
    try:
        print("Deleting prompt flow...")
        response = boto3_bedrock_agent_client.delete_flow(
            flowIdentifier=flow_id
        )
        print(json.dumps(response, indent=2, default=str))
        print("Prompt flow deletion completed.\n")
    except Exception as e:
        print(f"Error while deleting prompt flow: {e}\n")

def delete_prompt(prompt_id, prompt_type):
    """
    Deletes a specific prompt.

    Parameters:
        prompt_id (str): Identifier for the prompt to delete.
        prompt_type (str): Type of the prompt ('invoke' or 'eval') for descriptive purposes.
    """
    try:
        print(f"Deleting {prompt_type} prompt...")
        response = boto3_bedrock_agent_client.delete_prompt(
            promptIdentifier=prompt_id
        )
        print(json.dumps(response, indent=2, default=str))
        print(f"{prompt_type.capitalize()} prompt deletion completed.\n")
    except Exception as e:
        print(f"Error while deleting {prompt_type} prompt: {e}\n")

# Main Execution
try:

    delete_flow_alias(flowEvalId, flowEvalAliasId)
    delete_flow_version(flowEvalId, "1")
    delete_flow(flowEvalId)
    delete_prompt(promptInvokeId, "invoke")
    delete_prompt(promptEvalId, "eval")

except Exception as e:
    print(f"An unexpected error occurred during the cleanup process: {e}")

# End of NoteBook 

## Please ensure that you close the kernel after using this notebook to avoid any potential charges to your account.

## Process: Go to "Kernel" at top option. Choose "Shut Down Kernel". 
##### Refer https://docs.aws.amazon.com/sagemaker/latest/dg/studio-ui.html